# Контекстно-свободная грамматика для языка [Láadan](https://en.wikipedia.org/wiki/L%C3%A1adan)
Правила:
```
start: sentence
sentence: verb_phrase subject_phrase object_phrase? EVIDENCE_PARTICLE? EOS

verb_phrase: SPEECH_ACT_PARTICLE? TENSE_PARTICLE verb_form
verb_form: "me"? VERB
subject_phrase : NOUN | pronoun_marked | adjective_phrase
adjective_phrase: "wo" NOUN "wo" ADJ
pronoun_marked: PRONOUN_STEM "e" | "a" | "i"
object_phrase: NOUN "th"

EOS: "." | "!" | "?"
SPEECH_ACT_PARTICLE: "bíi" | "báa" | "bó" | "bóo" | "bé" | "bée"
TENSE_PARTICLE: "ril" | "eril" | "aril"
EVIDENCE_PARTICLE: "wa" | "wi" | "we" | "wáa" | "waá" | "wo" | "wóo"

NOUN: "bo" | "doyu" | "edan" | "rul"| "with" | "yu"
VERB: "áya" | "thi" | "yod" | "úuya"
ADJ: "áya"
PRONOUN_STEM: "l" | "n" | "b"

%import common.WS
%ignore /([wo]?<!) /
%ignore WS

```

Лаадан-русский словарь:
* Частицы речевого акта:
 * `bíi`: (немаркированное) повествовательная частица
 * `báa`: интеррогатив
 * `bó`: императив (как правило, к детям)
 * `bóo`: гортатив
 * `bé`: промиссив
 * `bée`: адмонитив
* Временные частицы для функциональных слов:
 * `ril`: настоящее время
 * `eril`: прошедшее время
 * `aril`: будущее время
* Эвиденциальные частицы:
 * `wa`: говорящий пережил описанное
 * `wi`: говорящий был свидетелем описанного
 * `we`: говорящий видел описанное во сне
 * `wáa`: надежный репортатив: воспринимается как правда, информация от надежного источника
 * `waá`: ненадежный репортатив: воспринимается как ложь, информация от ненадежного источника
 * `wo`: говорящий представляет описанное
 * `wóo`: говорящий не обладает знаниями о достоверности описанного
* Материальные слова:
 * `bo`: гора
 * `doyu`: яблоко
 * `edan`: лингвистика
 * `rul`: кот
 * `with`: человек, женщина (аналогично англ. *man*)
 * `yu`: фрукт
* Функциональные слова:
 * `a`: любовь/любить в отношении неодушевленных объектов
 * `áya`: красивый, быть красивым
 * `thi`: иметь
 * `yod`: есть
  * `úuya`: ранить, испытывать боль
* Местоимения:
 * к местоимениям **обязательно** прибавляется гласная `e/i/a`: `e`: немаркированно, `a`: о любимом человеке, `i`: об уважаемом человеке
 * `l-` : я
 * `n-`: ты
 * `b-`: он(а)

In [65]:
!pip install lark anytree

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 1.5 MB/s eta 0:00:00


In [69]:
from lark import Lark, Tree, Token
from lark import Transformer

from anytree import Node, RenderTree

In [61]:
laadan_grammar = """
start: sentence
sentence: verb_phrase subject_phrase object_phrase? EVIDENCE_PARTICLE? EOS

verb_phrase: SPEECH_ACT_PARTICLE? TENSE_PARTICLE verb_form
verb_form: "me"? VERB
subject_phrase : NOUN | pronoun_marked | adjective_phrase
adjective_phrase: "wo" NOUN "wo" ADJ
pronoun_marked: PRONOUN_STEM "e" | "a" | "i"
object_phrase: NOUN "th"

EOS: "." | "!" | "?"
SPEECH_ACT_PARTICLE: "bíi" | "báa" | "bó" | "bóo" | "bé" | "bée"
TENSE_PARTICLE: "ril" | "eril" | "aril"
EVIDENCE_PARTICLE: "wa" | "wi" | "we" | "wáa" | "waá" | "wo" | "wóo"

NOUN: "bo" | "doyu" | "edan" | "rul"| "with" | "yu"
VERB: "áya" | "thi" | "yod" | "úuya"
ADJ: "áya"
PRONOUN_STEM: "l" | "n" | "b"

%import common.WS
%ignore /([wo]?<!) /
%ignore WS
"""

laadan_parser = Lark(laadan_grammar, start='start')

In [62]:
sent = "bíi eril yod le doyuth wa." # я съела яблоко
tree = laadan_parser.parse(sent)
print(tree.pretty())

start
  sentence
    verb_phrase
      bíi
      eril
      verb_form	yod
    subject_phrase
      pronoun_marked	l
    object_phrase	doyu
    wa
    .



# Дерево

In [71]:
def tree_to_anytree(lark_node, parent=None):
    if isinstance(lark_node, Tree):
        node = Node(lark_node.data, parent=parent)
        for child in lark_node.children:
            tree_to_anytree(child, parent=node)
        return node
    elif isinstance(lark_node, Token):
        return Node(f"{lark_node.type}: {lark_node.value}", parent=parent)

anytree_root = tree_to_anytree(laadan_parser.parse("bíi eril yod le doyuth wa."))
for pre, _, node in RenderTree(anytree_root):
    print(f"{pre}{node.name}")

start
└── sentence
    ├── verb_phrase
    │   ├── SPEECH_ACT_PARTICLE: bíi
    │   ├── TENSE_PARTICLE: eril
    │   └── verb_form
    │       └── VERB: yod
    ├── subject_phrase
    │   └── pronoun_marked
    │       └── PRONOUN_STEM: l
    ├── object_phrase
    │   └── NOUN: doyu
    ├── EVIDENCE_PARTICLE: wa
    └── EOS: .


In [73]:
RenderTree(anytree_root)

RenderTree(Node('/start'), style=ContStyle(), childiter=<class 'list'>)